# Automated Metadata Extraction for Better Retrieval

In this tutorial, we show you how to perform automated metadata extraction for better retrieval results.

In [47]:
import nest_asyncio

nest_asyncio.apply()

import os
import openai

In [18]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [48]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.schema import MetadataMode

In [49]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", max_tokens=512)

We also show how to instantiate the `SummaryExtractor` and `QuestionsAnsweredExtractor`.

In [50]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=128, chunk_overlap=128)


metadata_extractor_1 = MetadataExtractor(
    extractors=[
        QuestionsAnsweredExtractor(questions=3, llm=llm),
    ],
    in_place = False
)

metadata_extractor = MetadataExtractor(
    extractors=[
        SummaryExtractor(summaries=["prev", "self", "next"], llm=llm),
        QuestionsAnsweredExtractor(questions=3, llm=llm),
    ],
    in_place = False
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    # metadata_extractor=metadata_extractor,
)

In [51]:
from llama_index import SimpleDirectoryReader

We first load in the Great Gatsby

In [52]:
docs = SimpleDirectoryReader(input_files=['../../../examples/gatsby/data/gatsby_part.txt']).load_data()

In [53]:
orig_nodes = node_parser.get_nodes_from_documents(docs)

In [54]:
# take just the first 8 nodes for testing
nodes = orig_nodes[:8]

In [55]:
print(nodes[3].get_content(metadata_mode="all"))

VII

It was when curiosity about Gatsby was at its highest that the lights
in his house failed to go on one Saturday night—and, as obscurely as
it had begun, his career as Trimalchio was over. Only gradually did I
become aware that the automobiles which turned expectantly into his
drive stayed for just a minute and then drove sulkily away. Wondering
if he were sick I went over to find out—an unfamiliar


In [56]:
# process nodes with metadata extractor
nodes_1 = metadata_extractor_1.process_nodes(nodes)

Extracting questions:   0%|          | 0/8 [00:00<?, ?it/s]

In [57]:
print(nodes_1[5].get_content(metadata_mode="all"))

[Excerpt from document]
questions_this_excerpt_can_answer: 1. What event marked the end of Gatsby's career as Trimalchio?
2. Why did the automobiles that arrived at Gatsby's house drive away after just a minute?
3. Who was the unfamiliar butler that the narrator encountered when they went to check on Gatsby?
Excerpt:
-----
VII

It was when curiosity about Gatsby was at its highest that the lights
in his house failed to go on one Saturday night—and, as obscurely as
it had begun, his career as Trimalchio was over. Only gradually did I
become aware that the automobiles which turned expectantly into his
drive stayed for just a minute and then drove sulkily away. Wondering
if he were sick I went over to find out—an unfamiliar butler with
-----


In [58]:
# 2nd pass: run summaries, and then metadata extractor

# process nodes with metadata extractor
nodes_2 = metadata_extractor.process_nodes(nodes)

Extracting summaries:   0%|          | 0/8 [00:00<?, ?it/s]

Extracting questions:   0%|          | 0/8 [00:00<?, ?it/s]

In [59]:
print(nodes_2[5].get_content(metadata_mode="all"))

[Excerpt from document]
prev_section_summary: In this section, the narrator mentions that curiosity about Gatsby is at its peak. However, one Saturday night, the lights in Gatsby's house do not turn on, and his career as Trimalchio abruptly ends. The narrator becomes aware that cars that arrive at Gatsby's house quickly leave, and they wonder if Gatsby is sick. The section also introduces an unfamiliar butler.
next_section_summary: In this section, the narrator mentions that curiosity about Gatsby is at its peak. However, one Saturday night, the lights in Gatsby's house do not turn on, and his career as Trimalchio abruptly ends. The narrator notices that cars that arrive at Gatsby's house quickly leave, and they wonder if Gatsby is sick. Additionally, an unfamiliar butler is mentioned.
section_summary: In this section, the narrator mentions that curiosity about Gatsby is at its peak. However, one Saturday night, the lights in Gatsby's house do not turn on, and his career as Trimalchio 

In [ ]:
print(nodes_2[6].get_content(metadata_mode="all"))

In [60]:
from llama_index import VectorStoreIndex
from llama_index.response.notebook_utils import display_source_node, display_response

In [ ]:
# try out different query engines

index0 = VectorStoreIndex(orig_nodes)
index1 = VectorStoreIndex(nodes_1 + orig_nodes[8:])
index2 = VectorStoreIndex(nodes_2 + orig_nodes[8:]) 

query_engine0 = index0.as_query_engine()
query_engine1 = index1.as_query_engine()
query_engine2 = index2.as_query_engine()

In [42]:
# query_str = "Can you describe the resemblance of Daisy's daughter to her or her father?" 
query_str = "How does Gatsby react to seeing Daisy's child for the first time?"

response0 = query_engine0.query(query_str)
response1 = query_engine1.query(query_str)
response2 = query_engine2.query(query_str)

In [43]:
response1.source_nodes[0].node.metadata

{'questions_this_excerpt_can_answer': "1. How does Daisy's behavior towards Gatsby differ from her behavior towards Nick?\n2. How does Gatsby react to seeing Daisy's child for the first time?\n3. How does Daisy describe her daughter's appearance and resemblance to her?"}

In [44]:
display_response(response0, source_length=1000, show_source=True)

**`Final Response:`** Gatsby reacts to seeing Daisy's child for the first time with surprise. He had never really believed in the child's existence before.

---

**`Source Node 1/2`**

**Node ID:** e9f405a1-fa51-452b-ac13-0fc3f728dba3<br>**Similarity:** 0.868435983578608<br>**Text:** you. You
absolute little dream.”

“Yes,” admitted the child calmly. “Aunt Jordan’s got on a white dress
too.”

“How do you like mother’s friends?” Daisy turned her around so that
she faced Gatsby. “Do you think they’re pretty?”

“Where’s Daddy?”

“She doesn’t look like her father,” explained Daisy. “She looks like
me. She’s got my hair and shape of the face.”

Daisy sat back upon the couch. The nurse took a step forward and held
out her hand.

“Come, Pammy.”

“Goodbye, sweetheart!”

With a reluctant backward glance the well-disciplined child held to
her nurse’s hand and was pulled out the door, just as Tom came back,
preceding four gin rickeys that clicked full of ice.

Gatsby took up his drink.

“They certainly look cool,” he said, with visible tension.

We drank in long, greedy swallows.

“I read somewhere that the sun’s getting hotter every year,” said Tom
genially. “It seems that pretty soon the earth’s going to fall into
the sun—or wait a minute—it’s just the opposite—the sun’s...<br>

---

**`Source Node 2/2`**

**Node ID:** 0500a508-aef8-4298-8b87-72d783c27b44<br>**Similarity:** 0.858142033308225<br>**Text:** us a cold drink,” cried Daisy.

As he left the room again she got up and went over to Gatsby and
pulled his face down, kissing him on the mouth.

“You know I love you,” she murmured.

“You forget there’s a lady present,” said Jordan.

Daisy looked around doubtfully.

“You kiss Nick too.”

“What a low, vulgar girl!”

“I don’t care!” cried Daisy, and began to clog on the brick fireplace.
Then she remembered the heat and sat down guiltily on the couch just
as a freshly laundered nurse leading a little girl came into the room.

“Bles-sed pre-cious,” she crooned, holding out her arms. “Come to your
own mother that loves you.”

The child, relinquished by the nurse, rushed across the room and
rooted shyly into her mother’s dress.

“The bles-sed pre-cious! Did mother get powder on your old yellowy
hair? Stand up now, and say—How-de-do.”

Gatsby and I in turn leaned down and took the small reluctant hand.
Afterward he kept looking at the child with surprise. I don’t think he
had ever really ...<br>

In [46]:
display_response(response1, source_length=1000, show_source=True)

**`Final Response:`** Gatsby reacts to seeing Daisy's child for the first time with surprise. He had never really believed in the child's existence before.

---

**`Source Node 1/2`**

**Node ID:** 3e7bc2f8-4815-4a4c-9d64-7f6ca6d82f0c<br>**Similarity:** 0.8943126827422523<br>**Text:** us a cold drink,” cried Daisy.

As he left the room again she got up and went over to Gatsby and
pulled his face down, kissing him on the mouth.

“You know I love you,” she murmured.

“You forget there’s a lady present,” said Jordan.

Daisy looked around doubtfully.

“You kiss Nick too.”

“What a low, vulgar girl!”

“I don’t care!” cried Daisy, and began to clog on the brick fireplace.
Then she remembered the heat and sat down guiltily on the couch just
as a freshly laundered nurse leading a little girl came into the room.

“Bles-sed pre-cious,” she crooned, holding out her arms. “Come to your
own mother that loves you.”

The child, relinquished by the nurse, rushed across the room and
rooted shyly into her mother’s dress.

“The bles-sed pre-cious! Did mother get powder on your old yellowy
hair? Stand up now, and say—How-de-do.”

Gatsby and I in turn leaned down and took the small reluctant hand.
Afterward he kept looking at the child with surprise. I don’t think he
had ever really ...<br>

---

**`Source Node 2/2`**

**Node ID:** 19895f2c-f258-45b7-89ef-be9fa1672241<br>**Similarity:** 0.8601995954458262<br>**Text:** you. You
absolute little dream.”

“Yes,” admitted the child calmly. “Aunt Jordan’s got on a white dress
too.”

“How do you like mother’s friends?” Daisy turned her around so that
she faced Gatsby. “Do you think they’re pretty?”

“Where’s Daddy?”

“She doesn’t look like her father,” explained Daisy. “She looks like
me. She’s got my hair and shape of the face.”

Daisy sat back upon the couch. The nurse took a step forward and held
out her hand.

“Come, Pammy.”

“Goodbye, sweetheart!”

With a reluctant backward glance the well-disciplined child held to
her nurse’s hand and was pulled out the door, just as Tom came back,
preceding four gin rickeys that clicked full of ice.

Gatsby took up his drink.

“They certainly look cool,” he said, with visible tension.

We drank in long, greedy swallows.

“I read somewhere that the sun’s getting hotter every year,” said Tom
genially. “It seems that pretty soon the earth’s going to fall into
the sun—or wait a minute—it’s just the opposite—the sun’s...<br>

In [21]:
query_str = (
    "When Daisy, Tom, and Gatsby all meet, Daisy eventually becomes overwhelmed and suggests going to town. "
    "The atmosphere beforehand is tense. Can you describe that?"
)

response0 = query_engine0.query(query_str)
response1 = query_engine1.query(query_str)
response2 = query_engine2.query(query_str)

In [24]:
print(str(response0))

The atmosphere before Daisy suggests going to town is described as hot and confused. Daisy is on the verge of tears and insists on going to town, while Tom tries to dismiss her concerns and tells her to forget about the heat. The tension is palpable as Daisy and Gatsby exchange glances and stare at each other, alone in space. The heat and the sense of confusion contribute to the tense atmosphere in the room.


In [34]:
for node in response0.source_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** fe89ee58-54dc-49a1-935b-c30b47a9d275<br>**Similarity:** 0.8772335973278944<br>**Text:** with them out to the veranda. On the green Sound, stagnant in
the heat, one small sail crawled slowly toward the fresher sea.
Gatsby’s eyes followed it momentarily; he raised his hand and pointed
across the bay.

“I’m right across from you.”

“So you are.”

Our eyes lifted over the rose-beds and the hot lawn and the weedy
refuse of the dog-days alongshore. Slowly the white wings of the boat
moved against the blue cool limit of the sky. Ahead lay the scalloped
ocean and the abounding blessed isles.

“There’s sport for you,” said Tom, nodding. “I’d like to be out there
with him for about an hour.”

We had luncheon in the dining-room, darkened too against the heat, and
drank down nervous gaiety with the cold ale.

“What’ll we do with ourselves this afternoon?” cried Daisy, “and the
day after that, and the next thirty years?”

“Don’t be morbid,” Jordan said. “Life starts all over again when it
gets crisp in the fall.”

“But it’s so hot,” insisted Daisy, on the verge of tears, “and
every...<br>

**Node ID:** be652beb-f8bd-45a3-9857-97d8c99396e9<br>**Similarity:** 0.8737805005250324<br>**Text:** and, though it was already four
o’clock, opening the windows admitted only a gust of hot shrubbery
from the Park. Daisy went to the mirror and stood with her back to us,
fixing her hair.

“It’s a swell suite,” whispered Jordan respectfully, and everyone
laughed.

“Open another window,” commanded Daisy, without turning around.

“There aren’t any more.”

“Well, we’d better telephone for an axe—”

“The thing to do is to forget about the heat,” said Tom impatiently.
“You make it ten times worse by crabbing about it.”

He unrolled the bottle of whisky from the towel and put it on the
table.

“Why not let her alone, old sport?” remarked Gatsby. “You’re the one
that wanted to come to town.”

There was a moment of silence. The telephone book slipped from its
nail and splashed to the floor, whereupon Jordan whispered, “Excuse
me”—but this time no one laughed.

“I’ll pick it up,” I offered.

“I’ve got it.” Gatsby examined the parted string, muttered “Hum!” in
an interested way, and tossed the...<br>

In [25]:
print(str(response1))

The atmosphere before Daisy suggests going to town is tense. There is a sense of unease and tension between the characters, particularly between Tom and Gatsby. Daisy's comment about Gatsby looking cool seems to further intensify the tension. Tom's reaction to Daisy's comment and his exchange with Gatsby further contribute to the tense atmosphere. Additionally, the oppressive heat adds to the discomfort and unease felt by the characters.


In [26]:
print(str(response2))

The atmosphere before Daisy suggests going to town is described as tense.
